## Resize

In [ ]:
import imgaug.augmenters as iaa
import imgaug as ia
import imageio
import numpy as np
import matplotlib.pyplot as plt

img_path = "Abyssinian_1"
img_dir = "Train/color/"
label_dir = "Train/label/"
image = imageio.imread(f"{img_dir}{img_path}.jpg")
mask = imageio.imread(f"{label_dir}{img_path}.png")
mask = mask[:,:,0]

# Define augmentation pipeline
im_resize_padding = iaa.Sequential([
    # Step 1: Pad to 1:1 aspect ratio (square) before resizing
    iaa.PadToAspectRatio(
        1.0,                     # Target aspect ratio (width/height)
        position="center",       # Center the image during padding
        pad_mode="constant",     # Pad with black (0) or use "edge"/"reflect"
        pad_cval=0               # Value used for padding
    ),
    # Step 2: Resize to 512x512 (now safe, aspect ratio is 1:1)
    iaa.Resize(
        512
    )
])

im_resize = iaa.Resize(512)

# Define augmentation pipeline
label_resize_padding = iaa.Sequential([
    # Step 1: Pad to 1:1 aspect ratio (square) before resizing
    iaa.PadToAspectRatio(
        1.0,                     # Target aspect ratio (width/height)
        position="center",       # Center the image during padding
        pad_mode="constant",     # Pad with black (0) or use "edge"/"reflect"
        pad_cval=0               # Value used for padding
    ),
    # Step 2: Resize to 512x512 (now safe, aspect ratio is 1:1)
    iaa.Resize(
        512,                     # Target size
        interpolation="nearest"   # For images ("nearest" for masks)
    )
])

label_resize = iaa.Resize(512, interpolation="nearest")



# Apply to image and mask
resized_image = aug.augment_image(image)
resized_mask = aug_mask.augment_image(mask)  # Remove channel dim (back to H, W)

resized_image_2 = resize.augment_image(image)

plt.imshow(resized_image_2)
plt.show()

## Rotation

In [ ]:
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# Convert mask to SegmentationMapsOnImage object
segmap = SegmentationMapsOnImage(mask, shape=image.shape)

# Define augmentation with black padding
aug = iaa.Affine(
    rotate=(45, 315),
    order=3,          # Image interpolation (cubic)
    mode="constant",  # Image padding mode
    cval=0,           # Image padding value (black)
    backend="cv2"
)

# Apply augmentation
augmented_image, augmented_segmap = aug(
    image=image,
    segmentation_maps=segmap
)

# Extract augmented mask
augmented_mask = augmented_segmap.get_arr()

plt.imshow(augmented_image)
plt.show()
plt.imshow(augmented_mask)
plt.show()

## Random Cropping

In [ ]:
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np

def random_square_crop_imgaug(image, mask, crop_biggest_square=False, random_state=None):
    """
    Crops a square from the image and a corresponding label mask
    using imgaug's built-in functions.

    If crop_biggest_square is True, it will crop the biggest possible square
    from the center of the image. Otherwise, it will crop a random square
    with a side length of 2/3 of the smallest edge of the image.

    Args:
        image (np.ndarray): The input image (H, W, C).
        mask (np.ndarray): The label mask (H, W).
        crop_biggest_square (bool): Whether to crop the biggest square
            from the center of the image. Defaults to False.
        random_state (None or int or imgaug.random.RNG or numpy.random.RandomState, optional):
            Random state to use for random operations. Defaults to None.

    Returns:
        tuple: A tuple containing the cropped image and the cropped mask.
    """

    if random_state is None:
        random_state = np.random.RandomState()  # Use numpy's RandomState by default
    elif isinstance(random_state, int):
        random_state = np.random.RandomState(random_state)
    elif isinstance(random_state, ia.random.RNG):
        pass  # Use the provided imgaug RNG directly
    elif isinstance(random_state, np.random.RandomState):
        pass  # Use the provided numpy RandomState directly
    else:
        raise ValueError("Invalid random_state.  Must be None, int, imgaug.random.RNG, or numpy.random.RandomState.")


    # Convert numpy RandomState to imgaug.random.RNG if needed
    if isinstance(random_state, np.random.RandomState):
        random_state = ia.random.RNG(random_state.randint(0, 10**6))

    height, width = image.shape[:2]

    if crop_biggest_square:
        cropper = iaa.CropToAspectRatio(1, "center")
        cropped_image = cropper.augment_image(image)
        cropped_mask = cropper.augment_image(mask)
    else:
        # Crop a random square with a side length of 2/3 of the smallest edge
        min_side = min(height, width)
        crop_size = int(min_side * (2/3))

        # Determine maximum possible top-left corner coordinates for the crop
        max_x = width - crop_size
        max_y = height - crop_size

        # Generate random top-left corner coordinates
        x1 = random_state.randint(0, max_x + 1)
        y1 = random_state.randint(0, max_y + 1)

        # Create a bounding box object representing the crop
        bbox = ia.BoundingBox(x1=x1, y1=y1, x2=x1 + crop_size, y2=y1 + crop_size)
        bbs = ia.BoundingBoxesOnImage([bbox], shape=image.shape)

        # Crop the image and mask based on the bounding box
        cropper = iaa.CropToFixedSize(width=crop_size, height=crop_size) # Crop and resize to the crop size
        cropped_image = cropper.augment_image(image[bbox.y1:bbox.y2, bbox.x1:bbox.x2])
        cropped_mask = cropper.augment_image(mask[bbox.y1:bbox.y2, bbox.x1:bbox.x2])


    return cropped_image, cropped_mask


# Crop the image and mask using the imgaug function
cropped_image, cropped_mask = random_square_crop_imgaug(image, mask, True)

plt.imshow(cropped_image)
plt.show()
plt.imshow(cropped_mask)
plt.show()

## Random masking

In [ ]:
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# Assuming 'image' is your input image array (H,W,C) 
# and 'label_mask' is your integer-label array (H,W)
seg_map = SegmentationMapsOnImage(mask, shape=image.shape)

# Define augmenters with CoarseDropout
aug = iaa.Sequential([
    iaa.CoarseDropout(p=0.05, size_percent=(1/8))
])

# Apply augmentation synchronously
images_aug_list = aug(
    images=[image]
)

# Retrieve augmented results
masked_image = images_aug_list[0]

# Visualize results
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2)
axs[0].imshow(masked_image)
axs[1].imshow(mask)
plt.show()

In [ ]:
import os
import shutil
import random

# Set seed for reproducibility
random.seed(42)

# Create train and validation directories
os.makedirs('Train/color', exist_ok=True)
os.makedirs('Train/label', exist_ok=True)
os.makedirs('Val/color', exist_ok=True)
os.makedirs('Val/label', exist_ok=True)

image_dir = 'TrainVal/color'
label_dir = 'TrainVal/label'

species_files = {}

# Collect image files with corresponding labels, grouped by species
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        label_filename = filename.replace('.jpg', '.png')
        label_path = os.path.join(label_dir, label_filename)
        if not os.path.exists(label_path):
            print(f"Skipping {filename} (label not found)")
            continue
        species = filename.split('_')[0]
        if species not in species_files:
            species_files[species] = []
        species_files[species].append(filename)

# Process each species to split and move files
for species, files in species_files.items():
    random.shuffle(files)
    split_idx = int(0.8 * len(files))
    train_files = files[:split_idx]
    val_files = files[split_idx:]
    
    # Move training data
    for file in train_files:
        # Move image
        src_img = os.path.join(image_dir, file)
        dst_img = os.path.join('Train/color', file)
        shutil.move(src_img, dst_img)
        # Move label
        label_file = file.replace('.jpg', '.png')
        src_label = os.path.join(label_dir, label_file)
        dst_label = os.path.join('Train/label', label_file)
        shutil.move(src_label, dst_label)
    
    # Move validation data
    for file in val_files:
        src_img = os.path.join(image_dir, file)
        dst_img = os.path.join('Val/color', file)
        shutil.move(src_img, dst_img)
        # Move label
        label_file = file.replace('.jpg', '.png')
        src_label = os.path.join(label_dir, label_file)
        dst_label = os.path.join('Val/label', label_file)
        shutil.move(src_label, dst_label)

print("Dataset split into Train and Val folders successfully.")

## Grayscale

In [ ]:
grayscale_aug = iaa.Grayscale(alpha=1.0, from_colorspace="RGB")

image_gray = grayscale_aug(image=image)

plt.imshow(image_gray)
plt.show()

## Laplace Noise

In [ ]:
laplace = iaa.AdditiveLaplaceNoise(scale=(0.1*255, 0.3*255), per_channel=True)
image_laplace = laplace(image=image)

plt.imshow(image_laplace)
plt.show()

In [ ]:
blur = iaa.AverageBlur(k=(12))

image_blur = blur(image=image)

plt.imshow(image_blur)
plt.show()


## Contrast

In [ ]:
contrast = iaa.LinearContrast((0.2, 0.6))

# Apply to an image (or batch)
image_contrast = contrast(image=image)

plt.imshow(image_contrast)
plt.show()

In [ ]:
from PIL import Image
import math

def combine_images(image1_path, image2_path, output_path=None):
    # Open images
    img1 = Image.open(image1_path)
    img2 = Image.open(image2_path)
    
    # Get dimensions
    w1, h1 = img1.size
    w2, h2 = img2.size
    
    # Determine orientations and longest edges
    def get_orientation(w, h):
        return 'portrait' if h > w else 'landscape'
    
    orientation1 = get_orientation(w1, h1)
    orientation2 = get_orientation(w2, h2)

    if orientation1 != orientation2:
        print("no merge")
        raise BaseException("no merge")
    
    # Determine concatenation direction
    if orientation1 == orientation2:
        direction = 'horizontal' if orientation1 == 'portrait' else 'vertical'
    else:
        le1 = max(w1, h1)
        le2 = max(w2, h2)
        direction = 'horizontal' if le1 > le2 and orientation1 == 'portrait' or le2 > le1 and orientation2 == 'portrait' else 'vertical'

    # Calculate scaling factors
    if direction == 'horizontal':
        # Scale to fit combined width = 512, height <= 512
        total_width = w1 + w2
        scale = min(512/total_width, 512/max(h1, h2))
    else:
        # Scale to fit combined height = 512, width <= 512
        total_height = h1 + h2
        scale = min(512/total_height, 512/max(w1, w2))

    # Resize both images proportionally
    def resize_img(img, original_w, original_h):
        new_w = math.floor(original_w * scale)
        new_h = math.floor(original_h * scale)
        return img.resize((new_w, new_h), Image.Resampling.LANCZOS)

    img1_resized = resize_img(img1, w1, h1)
    img2_resized = resize_img(img2, w2, h2)

    # Create combined image
    if direction == 'horizontal':
        combined_width = img1_resized.width + img2_resized.width
        combined_height = max(img1_resized.height, img2_resized.height)
        combined = Image.new('RGB', (combined_width, combined_height))
        combined.paste(img1_resized, (0, 0))
        combined.paste(img2_resized, (img1_resized.width, 0))
    else:
        combined_height = img1_resized.height + img2_resized.height
        combined_width = max(img1_resized.width, img2_resized.width)
        combined = Image.new('RGB', (combined_width, combined_height))
        combined.paste(img1_resized, (0, 0))
        combined.paste(img2_resized, (0, img1_resized.height))

    # Center in 512x512 canvas
    final_img = Image.new('RGB', (512, 512), (0, 0, 0))
    x_offset = (512 - combined.width) // 2
    y_offset = (512 - combined.height) // 2
    final_img.paste(combined, (x_offset, y_offset))

    if output_path:
        final_img.save(output_path)
    return final_img

# Usage
combined = combine_images("Train/label/beagle_154.png", "Train/label/beagle_145.png")

plt.imshow(combined)
plt.show()